In [1]:
import pickle
import pandas as pd
import os

import torch
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from tqdm.notebook import tqdm, trange
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
import wandb
from torch.utils.data.sampler import SubsetRandomSampler
import pprint
import matplotlib.pyplot as plt
import torch

from sklearn.preprocessing import MinMaxScaler
import math
from torch.profiler import profile, record_function, ProfilerActivity

from operator import itemgetter
import operator
from random import randint
# from rnn_classes import Dog, DogInput, Race, Races, GRUNet, smallGRUNet, smalll_lin_GRUNet, smalll_prelin_GRUNet
import rnn_classes
from raceDB import build_dataset, build_pred_dataset
import importlib
import datetime
from model_saver import model_saver, model_saver_wandb
import training_testing_gru
from torch.nn.utils.rnn import pack_padded_sequence, pack_sequence, pad_packed_sequence,pad_sequence, unpack_sequence, unpad_sequence

<frozen importlib._bootstrap>:228: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject


c:\Users\Nick\Documents\GitHub\grvmodel\Python\pytorch\New Model\LSTM


In [2]:
WANDB_NOTEBOOK_NAME = "GRU - v6.2.5.1"

In [3]:
if torch.cuda.is_available():
    device = torch.device("cuda:0")  # you can continue going on here, like cuda:1 cuda:2....etc.
    print("Running on the GPU")
else:
    device = torch.device("cpu")
    print("Running on the CPU")

Running on the GPU


In [4]:
torch.set_default_tensor_type(torch.FloatTensor)

In [5]:
def custom_l2(output, target):
    loss = torch.mean(abs(output-target), dim=1)
    return loss

In [6]:
def validation_CLE(x,y):
    loss_t = -torch.log(torch.exp(x)/torch.sum(torch.exp(x), dim=-1, keepdim=True))*y
    return loss_t

In [7]:
os.getcwd()
os.chdir(r"C:\Users\Nick\Documents\GitHub\grvmodel\Python\DATA")
date = datetime.datetime.strptime("2022-08-01", "%Y-%m-%d").date()
hidden_size = 32
states = ["NZ"]
data_file = 'gru_inputs_simple_v6_updated.fth'
raceDB = build_dataset(data_file, hidden_size ,state_filter=states, margin_type='boosted_sftmin',v6=True)
raceDB.create_new_weights_v2()
#raceDB.adjust_weights({"Dapto":10, "Gunnedah":10})

[['dist_last__1', 'box_last__1', 'speed_avg_1', 'split_speed_avg_1', 'split_margin_avg_1', 'margin_avg_1', 'RunHomeTime_1', 'run_home_speed_1', 'first_out_avg_1', 'pos_out_avg_1', 'post_change_avg_1', 'races_1', 'wins_1', 'wins_last_1', 'weight_', 'min_time_', 'min_split_time_', 'last_start_price', 'last_start_prob']]
(1273739, 24)
25
Latest date = 2023-06-14 00:00:00
size after state filter (134082, 26)
(134082, 26)
Latest date = 2023-06-13 00:00:00
num_features_per_dog=25


  0%|          | 0/3649 [00:00<?, ?it/s]c:\Users\Nick\.conda\envs\PYTORCH\lib\site-packages\tqdm\std.py:1195: FutureWarning: In a future version of pandas, a length 1 tuple will be returned when iterating over a groupby with a grouper equal to a list of length 1. Don't supply a list with a single grouper to avoid this warning.
  for obj in iterable:
  0%|          | 0/17773 [00:00<?, ?it/s]c:\Users\Nick\.conda\envs\PYTORCH\lib\site-packages\tqdm\std.py:1195: FutureWarning: In a future version of pandas, a length 1 tuple will be returned when iterating over a groupby with a grouper equal to a list of length 1. Don't supply a list with a single grouper to avoid this warning.
  for obj in iterable:
100%|██████████| 17773/17773 [00:49<00:00, 359.45it/s]


number of races = 17773, number of unique dogs = 3650
0            (Auckland, 318.0)
1            (Auckland, 318.0)
2            (Auckland, 527.0)
3            (Auckland, 318.0)
4            (Auckland, 318.0)
                 ...          
17768    (Christchurch, 295.0)
17769    (Christchurch, 520.0)
17770    (Christchurch, 295.0)
17771    (Christchurch, 295.0)
17772    (Christchurch, 295.0)
Length: 17773, dtype: object


In [8]:
date = datetime.datetime.strptime("2022-08-01", "%Y-%m-%d").date()
raceDB.create_test_split_date(date)

Train examples 13565, Test examples 4208


In [9]:
raceDB.create_dogs_test_split_date()

100%|██████████| 3649/3649 [00:00<00:00, 24467.67it/s]


In [10]:
raceDB.attach_races_to_dog_inputv2()

100%|██████████| 3650/3650 [00:02<00:00, 1468.66it/s]


In [11]:
raceDB.get_dog_test([1,2])[0][0].stats

tensor([ 4.0000,  0.0000,  1.0000,  0.0000, 24.5000,  2.0000, -1.0000, -1.0000,
        -1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000,
        -1.0000,  0.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000,
        -1.0000], device='cuda:0', dtype=torch.float64)

In [12]:
raceDB.create_batches(batch_days=180)

[(datetime.date(2019, 12, 1), datetime.date(2020, 5, 29)), (datetime.date(2020, 5, 29), datetime.date(2020, 11, 25)), (datetime.date(2020, 11, 25), datetime.date(2021, 5, 24)), (datetime.date(2021, 5, 24), datetime.date(2021, 11, 20)), (datetime.date(2021, 11, 20), datetime.date(2022, 5, 19)), (datetime.date(2022, 5, 19), datetime.date(2022, 7, 31))]
2020-05-29
2020-11-25
2021-05-24
2021-11-20
2022-05-19
2022-07-31
Train examples [2201, 2826, 2680, 2396, 2444, 1001]


100%|██████████| 6/6 [00:05<00:00,  1.06it/s]


Train examples [1557, 1594, 1489, 1492, 1400, 1111]
Train examples [1557, 1594, 1489, 1492, 1400, 1111]
Train examples [2201, 2826, 2680, 2396, 2444, 1001]
Train examples [2201, 2826, 2680, 2396, 2444, 1001]


In [13]:
def closure(optimizer, criterion, outs, classes):
    optimizer.zero_grad()
    loss = nn.functional.mse_loss(outs, classes)
    loss.backward()
    return loss

def model_pipeline(my_dataset=raceDB,config=None,prev_model=None, sweep=True, model_state_dict=None, prev_model_file=None, prev_model_version='450'):
    if my_dataset:
      dataset = my_dataset    
    else:
      dataset = raceDB
    # tell wandb to get started
    with wandb.init(project="NEW GRU", config=config):
      #  access all HPs through wandb.config, so logging matches execution!
      wandb.define_metric("loss_val", summary="min")
      wandb.define_metric("accuracy", summary="max")
      wandb.define_metric("ROI < 30", summary="max")
      wandb.define_metric("relu roi", summary="max")
      
      config = wandb.config
      pprint.pprint(config)
      pprint.pprint(config.epochs)
      print(config)
      # input_size = raceDB.get_race_input([0,1])[0].full_input.shape[0] #create fix so messy
      print(config.input_type)

      if 'batch_days' in config.keys():
        raceDB.create_batches(batch_days=config['batch_days'])

      if config.input_type == 'full':
        print('here1')
        input_size = raceDB.batches['packed_x'][0].data[0].shape[0]
      elif config.input_type == 'basic':
        print('here')
        raceDB.batches['packed_x'] = raceDB.batches['packed_x_basic']
        raceDB.batches['packed_y'] = raceDB.batches['packed_y_basic']
        raceDB.batches['num_batches'] = len(raceDB.batches['packed_x'])
        input_size = raceDB.batches['packed_x'][0].data[0].shape[0]

      print(f"{input_size=}")





      raceDB.reset_hidden(num_layers=config['num_layers'], hidden_size=config['hidden_size'])
      model = rnn_classes.ReluNetv1(input_size,config['hidden_size'], num_layers=config['num_layers'],fc0_size=config['f0_layer_size'], fc1_size=config['f1_layer_size'])

      if model_state_dict:
        model.load_state_dict(model_state_dict)
      if prev_model_file!=None:
        model_name = prev_model_file
        model_loc = f"C:/Users/Nick/Documents/GitHub/grvmodel/Python/pytorch/New Model/savedmodel/{model_name}/{model_name}_{prev_model_version}.pt"
        model_data = torch.load(model_loc,map_location=torch.device('cuda:0'))
        # model_data['model_state_dict'].pop('gru.weight_ih_l0')
        # model_data['model_state_dict'].pop('layer_norm.weight')
        # model_data['model_state_dict'].pop('layer_norm.bias')
        model.load_state_dict(model_data['model_state_dict'], strict=False)
        config['parent model'] = prev_model_file
        model = model.to(device)
        optimizer = optim.AdamW(model.parameters(), lr=config['learning_rate'])
        optimizer.load_state_dict(model_data['optim'])
        # optimizer.to(device)
      else:
        optimizer = optim.Adam(model.parameters(), lr=config['learning_rate'])

      raceDB.to_cuda()
      optimizer = optim.AdamW(model.parameters(), lr=config['learning_rate'])
      criterion = nn.CrossEntropyLoss(reduction='none', label_smoothing=config['label_smoothing'])

      model = model.to(device)

      # optimizer = optimizer.to(device)
      print(model)

      # and use them to train the model
      try:
        # train_double_loss_regular(model, dataset, criterion, optimizer, scheduler, config, crit2=custom_l2)
        training_testing_gru.train_regular_v3(model, dataset, criterion, optimizer, 'na', config)
      except (KeyboardInterrupt) as error:
        print(error)
        print("finished Early")
        
      # dataset.create_hidden_states_dict()
      raceDB.create_hidden_states_dict_v2()
      model_saver_wandb(model, optimizer, 450, 0.1, raceDB.hidden_states_dict_gru_v6,raceDB.train_hidden_dict , model_name="long nsw new  22000 RUN")
      if sweep:
        # raceDB.reset_all_lstm_states
        raceDB.reset_hidden()
    


    # and test its final performance
    #test(model, test_loader)

    return (model,dataset, optimizer)

In [14]:
len(raceDB.raceIDs)
wandb_config_static = {'hidden_size':128,
                       'stats':raceDB.stats_cols,
                       'races':states,
                       'datafile':data_file,
                       'latest_date':raceDB.latest_date,
                       'input_type':'basic',
                       'num_layers':2,
                       'batch_size': 750,
                       'dropout': 0.3,
                       'epochs': 10_000,
                       'learning_rate': 0.00005,
                       'label_smoothing':0.01,
                       'optimizer': 'adamW',
                       'f0_layer_size':128,
                       'f1_layer_size':64,
                       'batch_days':180}

In [15]:
importlib.reload(training_testing_gru)
importlib.reload(rnn_classes)

C:\Users\Nick\Documents\GitHub\grvmodel\Python\DATA


<module 'rnn_classes' from 'c:\\Users\\Nick\\Documents\\GitHub\\grvmodel\\Python\\pytorch\\New Model\\LSTM\\rnn_classes.py'>

In [16]:
raceDB.reset_hidden(num_layers=wandb_config_static['num_layers'], hidden_size=wandb_config_static['hidden_size'])

In [17]:
torch.cuda.empty_cache()

In [18]:
# for race in raceDB.racesDict.values():
#     for dog in race.dogs:
#         try:
#             del dog.stats
#         except:
#             pass

# torch.cuda.empty_cache()

In [19]:
(model,dataset, optimizer) = model_pipeline(raceDB,config=wandb_config_static,sweep=False)

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: nickojelly. Use `wandb login --relogin` to force relogin


{'hidden_size': 128, 'stats': "[['dist_last__1', 'box_last__1', 'speed_avg_1', 'split_speed_avg_1', 'split_margin_avg_1', 'margin_avg_1', 'RunHomeTime_1', 'run_home_speed_1', 'first_out_avg_1', 'pos_out_avg_1', 'post_change_avg_1', 'races_1', 'wins_1', 'wins_last_1', 'weight_', 'min_time_', 'min_split_time_', 'last_start_price', 'last_start_prob']]", 'races': ['NZ'], 'datafile': 'gru_inputs_simple_v9.fth', 'latest_date': '2023-06-13T00:00:00', 'input_type': 'basic', 'num_layers': 2, 'batch_size': 750, 'dropout': 0.3, 'epochs': 10000, 'learning_rate': 5e-05, 'label_smoothing': 0.01, 'optimizer': 'adamW', 'f0_layer_size': 128, 'f1_layer_size': 64, 'batch_days': 180}
10000
{'hidden_size': 128, 'stats': "[['dist_last__1', 'box_last__1', 'speed_avg_1', 'split_speed_avg_1', 'split_margin_avg_1', 'margin_avg_1', 'RunHomeTime_1', 'run_home_speed_1', 'first_out_avg_1', 'pos_out_avg_1', 'post_change_avg_1', 'races_1', 'wins_1', 'wins_last_1', 'weight_', 'min_time_', 'min_split_time_', 'last_star

100%|██████████| 6/6 [00:05<00:00,  1.05it/s]


Train examples [1557, 1594, 1489, 1492, 1400, 1111]
Train examples [1557, 1594, 1489, 1492, 1400, 1111]
Train examples [2201, 2826, 2680, 2396, 2444, 1001]
Train examples [2201, 2826, 2680, 2396, 2444, 1001]
here
input_size=25
ReluNetv1(
  (gru): GRU(25, 128, num_layers=2, dropout=0.3)
  (relu0): ReLU()
  (fc0): Linear(in_features=1094, out_features=1094, bias=True)
  (layer_norm): BatchNorm1d(25, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (rl1): ReLU()
  (drop1): Dropout(p=0.3, inplace=False)
  (fc1): Linear(in_features=1094, out_features=128, bias=True)
  (rl2): ReLU()
  (drop2): Dropout(p=0.3, inplace=False)
  (fc2): Linear(in_features=128, out_features=64, bias=True)
  (rl3): ReLU()
  (drop3): Dropout(p=0.3, inplace=False)
  (fc3): Linear(in_features=64, out_features=8, bias=True)
  (output_fn): Identity()
)


  0%|          | 0/10000 [00:00<?, ?it/s]

created path


 55%|█████▍    | 5457/10000 [11:11:33<9:19:04,  7.38s/it] 



finished Early


ROI < 30,▂▄▆▆▆▆▅▅▅████▆▆▅▄▄▃▂▄▂▃▁▂▂▂▃▃▂▂▃▃▂▄▃▄▃▃▄
ROI < 30 2,▁▄▆▆▅▄▄▅▅██▇▆▆▆▅▄▃▄▃▄▂▃▁▂▁▁▂▂▂▂▃▂▂▃▃▂▂▂▃
accuracy,▁▆██▇▇▆▆▆▆▆▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅
accuracy2,▁▆██▇▇▆▆▆▆▆▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅
epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███
flat_simple,▅▁▆█▇▃▅▅▆▇█▅▄▄▆▅▄▄▆▅▆▅▅▃▄▃▃▄▄▄▄▅▄▃▅▅▃▄▄▅
loss_1,██▇▇▇▂▁▁▁▃▃▃▃▃▃▃▃▃▁▁▁▁▃▂▂▂▂▂▂▂▂▁▁▁▁▂▂▂▂▂
loss_val,▃▂▁▁▁▁▂▃▃▄▄▅▅▅▅▆▆▆▆▆▆▆▆▇▇▇▇▇▇▇▇█▇███████
multibet outlay < 30,▁▁▁▁▂▂▃▄▄▅▆▆▆▇▇▇▇▇▇▇▇▇▇▇▇▇▇█████████████
multibet profit,▅▅▇▇▇▆▄▃▃▆▇▇▇▇▆▆▄▄▄▃▅▃▃▁▄▄▂▅▅▄▄▆▅▅▇▆▆▆▆█
multibet profit < 30,▄▅▅▅▅▅▅▅▅▇▇██▇▆▆▄▄▃▃▄▂▃▁▃▂▂▃▃▂▂▃▃▂▄▃▄▃▃▄


In [ ]:
raceDB.batches['num_batches']

In [ ]:
len(raceDB.batches['packed_x'])

In [ ]:
model_name='kind-wood-610'
prev_model_version=1300
model_loc = f"C:/Users/Nick/Documents/GitHub/grvmodel/Python/pytorch/New Model/savedmodel/{model_name}/{model_name}_{prev_model_version}.pt"
data = torch.load(model_loc,map_location=torch.device('cuda:0'))

In [ ]:
data['model_state_dict'].pop('gru.weight_ih_l0')
data['layer_norm.weight'].pop('gru.weight_ih_l0')
data['layer_norm.bias'].pop('gru.weight_ih_l0')

In [18]:
sweep_config = {"method": "grid"}

metric = {"name": "ROI < 30", "goal": "maximize"}

sweep_config["metric"] = metric


parameters_dict = {
    "optimizer": {"value": "adamW"},
    "batch_days": {"values": [90,180,365,550,10000]},
    "f0_layer_size": {"values": [128]},
    "f1_layer_size": {"values": [64]},
    "dropout": {"values": [0.3]},
    "input_type": {"values": ['basic']},
    "num_layers": {"values": [2]},
    'hidden_size':{'values':[128]},
    "len_data": {"value": len(raceDB.raceIDs)},
    "stats":{"value": raceDB.stats_cols},
    "races":{"value": states},
    "batch_size":{"value": 10},
    # "hidden_size": {"value":hidden_size}
}

sweep_config["parameters"] = parameters_dict

parameters_dict.update(
    {
        "epochs": {"values": [500]},
        "validation_split": {"value": 0.1},
        "learning_rate": {"values": [0.0003, 0.0001]},
        "label_smoothing": {"values": [0.01,0.0]},
        "loss": {
            "values": [ "CEL"],
            # "values": ["Huber", "MSE", "L1", "BCE", "Custom", "KL"]
            # 'value': 'l1_custom'
        },
        # "num_layers": {"values": [2]},
    }
)

# parameters_dict.update(
#     {
#         "learning_rate": {
#             # a flat distribution between 0 and 0.1
#             "distribution": "uniform",
#             "min": 0.0003,
#             "max": 0.0003,
#         },
#     }
# )

import pprint

pprint.pprint(sweep_config)


sweep_config

{'method': 'grid',
 'metric': {'goal': 'maximize', 'name': 'ROI < 30'},
 'parameters': {'batch_days': {'values': [90, 180, 365, 550, 10000]},
                'batch_size': {'value': 10},
                'dropout': {'values': [0.3]},
                'epochs': {'values': [500]},
                'f0_layer_size': {'values': [128]},
                'f1_layer_size': {'values': [64]},
                'hidden_size': {'values': [128]},
                'input_type': {'values': ['basic']},
                'label_smoothing': {'values': [0.01, 0.0]},
                'learning_rate': {'values': [0.0003, 0.0001]},
                'len_data': {'value': 17498},
                'loss': {'values': ['CEL']},
                'num_layers': {'values': [2]},
                'optimizer': {'value': 'adamW'},
                'races': {'value': ['NZ']},
                'stats': {'value': "[['dist_last__1', 'box_last__1', "
                                   "'speed_avg_1', 'split_speed_v1_1', "
                  

{'method': 'grid',
 'metric': {'name': 'ROI < 30', 'goal': 'maximize'},
 'parameters': {'optimizer': {'value': 'adamW'},
  'batch_days': {'values': [90, 180, 365, 550, 10000]},
  'f0_layer_size': {'values': [128]},
  'f1_layer_size': {'values': [64]},
  'dropout': {'values': [0.3]},
  'input_type': {'values': ['basic']},
  'num_layers': {'values': [2]},
  'hidden_size': {'values': [128]},
  'len_data': {'value': 17498},
  'stats': {'value': "[['dist_last__1', 'box_last__1', 'speed_avg_1', 'split_speed_v1_1', 'split_speed_avg_1', 'split_margin_avg_1', 'margin_avg_1', 'RunHomeTime_1', 'run_home_speed_1', 'run_home_speed_v1_1', 'first_out_avg_1', 'pos_out_avg_1', 'post_change_avg_1', 'races_1', 'wins_1', 'wins_last_1', 'weight_', 'min_time_', 'min_split_time_', 'min_split_time_v1', 'last_start_price', 'last_start_prob']]"},
  'races': {'value': ['NZ']},
  'batch_size': {'value': 10},
  'epochs': {'values': [500]},
  'validation_split': {'value': 0.1},
  'learning_rate': {'values': [0.0003

In [ ]:
wandb.agent('38qbqfbs', function=model_pipeline, count=100)

In [19]:
sweep_id = wandb.sweep(sweep_config, project="NEW GRU-sweeps")
CUDA_LAUNCH_BLOCKING=1
wandb.agent(sweep_id, function=model_pipeline, count=100)

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


Create sweep with ID: 9vayinwt
Sweep URL: https://wandb.ai/nickojelly/NEW%20GRU-sweeps/sweeps/9vayinwt


wandb: Agent Starting Run: plnf6mip with config:
wandb: 	batch_days: 90
wandb: 	batch_size: 10
wandb: 	dropout: 0.3
wandb: 	epochs: 500
wandb: 	f0_layer_size: 128
wandb: 	f1_layer_size: 64
wandb: 	hidden_size: 128
wandb: 	input_type: basic
wandb: 	label_smoothing: 0.01
wandb: 	learning_rate: 0.0003
wandb: 	len_data: 17498
wandb: 	loss: CEL
wandb: 	num_layers: 2
wandb: 	optimizer: adamW
wandb: 	races: ['NZ']
wandb: 	stats: [['dist_last__1', 'box_last__1', 'speed_avg_1', 'split_speed_v1_1', 'split_speed_avg_1', 'split_margin_avg_1', 'margin_avg_1', 'RunHomeTime_1', 'run_home_speed_1', 'run_home_speed_v1_1', 'first_out_avg_1', 'pos_out_avg_1', 'post_change_avg_1', 'races_1', 'wins_1', 'wins_last_1', 'weight_', 'min_time_', 'min_split_time_', 'min_split_time_v1', 'last_start_price', 'last_start_prob']]
wandb: 	validation_split: 0.1
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently lo

{'batch_days': 90, 'batch_size': 10, 'dropout': 0.3, 'epochs': 500, 'f0_layer_size': 128, 'f1_layer_size': 64, 'hidden_size': 128, 'input_type': 'basic', 'label_smoothing': 0.01, 'learning_rate': 0.0003, 'len_data': 17498, 'loss': 'CEL', 'num_layers': 2, 'optimizer': 'adamW', 'races': ['NZ'], 'stats': "[['dist_last__1', 'box_last__1', 'speed_avg_1', 'split_speed_v1_1', 'split_speed_avg_1', 'split_margin_avg_1', 'margin_avg_1', 'RunHomeTime_1', 'run_home_speed_1', 'run_home_speed_v1_1', 'first_out_avg_1', 'pos_out_avg_1', 'post_change_avg_1', 'races_1', 'wins_1', 'wins_last_1', 'weight_', 'min_time_', 'min_split_time_', 'min_split_time_v1', 'last_start_price', 'last_start_prob']]", 'validation_split': 0.1}
500
{'batch_days': 90, 'batch_size': 10, 'dropout': 0.3, 'epochs': 500, 'f0_layer_size': 128, 'f1_layer_size': 64, 'hidden_size': 128, 'input_type': 'basic', 'label_smoothing': 0.01, 'learning_rate': 0.0003, 'len_data': 17498, 'loss': 'CEL', 'num_layers': 2, 'optimizer': 'adamW', 'rac

100%|██████████| 11/11 [00:05<00:00,  1.98it/s]


...

  0%|          | 0/500 [00:00<?, ?it/s]

created path


100%|██████████| 500/500 [47:24<00:00,  5.69s/it]


ROI < 30,▃▃▄▄▁▄▄▆▆▆▆▆▇▇▇▇██▆▆▆▇▇▆▆▆▇▇▆▆▆▇▇▇▆▆▆▆▆▆
ROI < 30 2,▃▃▄▄▁▅▅▇▇▇▇▇▇██▇██▇▇▆▇▇▆▆▆▇▇▆▅▅▆▇▇▆▆▆▆▆▆
accuracy,▁▁▁▁▄▇▇██████▇▇▇▇▇▇▇▇▆▆▆▆▆▆▆▅▅▅▅▅▅▅▅▅▅▅▅
accuracy2,▁▁▄▇████▇▇▇▇▇▆▆▆▆▅▅▅▅▅▅▅▅
epoch,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
flat_simple,▆▆▆▆▁▃▃▅▇▇▆▆▆▅▅▆▆▆██▄▅▅▂▁▁▇▇▃▇▇▆▅▅▆▆▃▅▅▃
loss_1,█████▇▇▇▇▇▆▆▆▆▆▅▆▆▆▄▄▅▅▄▂▃▃▃▃▂▂▂▂▁▁▂▂▂▁▁
loss_val,▄▄▄▄▃▂▂▁▁▁▁▁▁▁▁▁▁▁▂▂▂▃▃▄▄▄▅▅▆▆▆▇▇▇██████
multibet outlay < 30,▂▂▂▂▁▁▁▁▁▁▁▁▂▂▂▂▃▃▃▃▄▅▅▆▆▆▇▇▇▇▇█████████
multibet profit,▃▃▂▂▁▄▄▅▅▅▄▄▄▄▄▄▅▅▄▄▅▆▆▄▄▄▇▇▅▇▇█▇▇▆▆▆▃▃▆
multibet profit < 30,▂▂▃▃▁▃▃▅▅▅▅▅▆▆▆▆██▆▆▆▇▇▅▅▅▇▇▅▅▅▇██▇▇▆▅▅▇


wandb: Ctrl + C detected. Stopping sweep.
